In [1]:
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

In [2]:
# !pip install labelme

In [3]:
import json
from labelme.utils.shape import labelme_shapes_to_label

In [4]:
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
def checkpath(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
def get_dir_json_list(root_dir):
    jsons_files = []
    for json_N in os.listdir(root_dir):
        if json_N[-1] == 'n':
            jsons_files += [os.path.join(root_dir, json_N)]
    return jsons_files

In [7]:
def get_dict(root_dir, image_DIR):

    json_list = get_dir_json_list(root_dir)

    dict_all = {}
    for json_path in json_list:
        dir,file = os.path.split(json_path)
        file_name = file.split('.')[0]
        image_path = os.path.join(image_DIR, file_name + '.jpg')
        dict_all[image_path] = json_path
    return dict_all

In [8]:
def kakunin_size(size):
  if size==1080:
    x = 420
    y = 0
    w = 1080
    h = 1080
  elif size==488:
    x = 116
    y = 0
    w = 488
    h = 488
  return x, y, w, h

In [9]:
def mask_process(masks_dir, image_DIR, error_list):
    dict_ = get_dict(masks_dir, image_DIR)

    for image_path in dict_:
        mask = []
        class_id = []
        key_ = []

        image = cv2.imread(image_path)
        json_path = dict_[image_path]
        print('looking at ' + str(image_path))
        # print('looking at ' + str(json_path))
        x, y, w, h = kakunin_size(image.shape[0])


        data = json.load(open(json_path))
        try:
            lbl, lbl_names = labelme_shapes_to_label(image.shape, data['shapes'])
        except AssertionError:
            error_list += [json_path]
            print('Got an error in ' + json_path)
            print('Jumping over!!')
            continue

        for i in range(1, len(lbl_names)):  # 跳过第一个class（因为0默认为背景,跳过不取！）
            key = [k for k, v in lbl_names.items() if v == i][0]
            mask.append((lbl == i).astype(np.uint8))  # 举例：当解析出像素值为1，此时对应第一个mask 为0、1组成的（0为背景，1为对象）
            class_id.append(i)  # mask与class_id 对应记录保存
            key_.append(key)
        mask = np.asarray(mask, np.uint8)
        mask = np.transpose(np.asarray(mask, np.uint8), [1, 2, 0])
        image_name = os.path.basename(image_path).split('.')[0]
        dir_ = os.path.dirname(image_path)
        for i in range(0, len(class_id)):
            image_name_ = "{}_mask_{}_{}.jpg".format(image_name,key_[i],i)
            dir_path =  os.path.join(dir_, '../mask',key_[i]) # 构建保存缺陷的文件夹 key_[i]为缺陷名称，i为缺陷ID
            checkpath(dir_path)
            image_path_ = os.path.join(dir_path,image_name_)
            retval, im_at_fixed = cv2.threshold(mask[:,:,i], 0, 255, cv2.THRESH_BINARY)
            
            im_at_fixed = im_at_fixed[y:y+h, x:x+w]

            cv2.imwrite(image_path_, im_at_fixed)

    with open(os.path.join(masks_dir, '../mask/' + masks_dir.split('/')[-1] + '_ERROR_LIST.txt'), 'w') as f:
        for get_error in error_list:
            f.write(str(get_error) + '\n')
    return error_list

In [10]:
def convert_jpg_to_unet(json_dir, TYPE):

  jpg_dir = json_dir.replace('json', '') + 'jpg_CNN'
    
  jpgs_DIR = os.path.join(jpg_dir, '../mask/jpgs')
  checkpath(jpgs_DIR)

  for jpg_N in os.listdir(os.path.join(jpg_dir, '../mask/frontglottis')):
    if jpg_N[-1] == 'g':
      if (jpg_N.split('.')[0])[-1] != '0':
        print('Got a bad news :(')
      jpgs_name = jpg_N.replace('_mask_frontglottis_0', '')
      img = cv2.imread(os.path.join(jpg_dir, jpgs_name))
      filename = os.path.join(jpgs_DIR, jpgs_name)
      print('facing ' + filename)

      x, y, w, h = kakunin_size(img.shape[0])
      img = img[y:y+h, x:x+w]

      cv2.imwrite(filename, img)

  memo = os.path.join(jpg_dir, '../mask/'+str(w))
  checkpath(memo)

  type_ = os.path.join(jpg_dir, '../mask/'+TYPE)
  checkpath(type_)

In [33]:
# root_dir = '/content/drive/My Drive/09_research_main/lab_01/data/dataP/test'

# root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL'
# root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R'
root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_L'
# root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RSLN_R'
# root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RSLN_L'

In [45]:
DIR_list = []

for get_folder_name in os.listdir(root_dir):
  watching_this_dir = os.path.join(root_dir, get_folder_name)
  DIR_list += [watching_this_dir]

print(len(DIR_list))
print(DIR_list)

8
['/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/組21_RLN_G1-17_388775A_M_R_72_201090212', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/組20_RLN_data 163_2489652__M_R_58_20180323', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/組13_RLN_data 120_1366274_F_R_27_20161111', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/組8_RLN_110_1279067_F_R_66_20160708', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組3_RLN_data_9_1540626_F_R_50_20180504', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組14_RLN_G1-18_1683105D_M_R_56_20190124', '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/組15_RLN_G1-22_1564057A_F_R_79_20211111']


In [63]:
# main

# data_checker(root_dir)

TYPE = 'RLN_R'
error_list = []


this_dir = DIR_list[4]

print('\nNow processing in ' + this_dir + '\n')

for get_folder in os.listdir(this_dir):
  if get_folder[-1] == 'n':
    processing_dir = os.path.join(this_dir, get_folder)

    for get_jpgs_folder in os.listdir(this_dir):
      if get_jpgs_folder[-1] == 'N':
        image_dir = os.path.join(this_dir, get_jpgs_folder)

        error_list += [mask_process(processing_dir, image_dir, error_list)]

        print('\nMask process Completed')

        convert_jpg_to_unet(processing_dir, TYPE)

        print('\nProcess Completed in ' + this_dir + '\n')


print('I found errors in :')
print(error_list)

print('\n\nSuccessfully Completed!!!\n\n')

[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecat


Now processing in /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220

looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_927.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_837.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_779.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_851.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_236046

[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecat

looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_878.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_758.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_789.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_802.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_859.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_d

[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecat

...

looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_905.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_902.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_891.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_937.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_858.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_d

[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecat

looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_895.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_818.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_867.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_870.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_829.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_d

[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.
[WARNING] shape:labelme_shapes_to_label:80 - labelme_shapes_to_label is deprecated, so please use shapes_to_label.


looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_897.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_768.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_783.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_771.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_data_15_2360468_M_R_57_20131220/組1_RLN_data_15_2360468_M_R_57_20131220_jpg_CNN/image_800.jpg
looking at /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/RLN_R/_組1_RLN_d

In [ ]:
# # main

# # data_checker(root_dir)

# TYPE = 'NORMAL'
# error_list = []

# for get_folder_name in os.listdir(root_dir):
#   watching_this_dir = os.path.join(root_dir, get_folder_name)
#   print('\nNow processing in ' + watching_this_dir + '\n')

#   for get_folder in os.listdir(watching_this_dir):
#     if get_folder[-1] == 'n':
#       processing_dir = os.path.join(watching_this_dir, get_folder)

#       error_list += [mask_process(processing_dir, error_list)]

#       print('\nMask process Completed')

#       convert_jpg_to_unet(processing_dir, TYPE)

#       print('\nProcess Completed in ' + watching_this_dir + '\n')

# print('\n\nSuccessfully Completed!!!\n\n')

# print('I found errors in ' + error_list)


In [ ]:
error_list

In [ ]:
123=12

In [ ]:
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/483_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/484_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/485_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/486_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/487_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/488_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/489_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/490_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/491_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/492_mask_frontglottis_1.jpg
# facing /home/yasaisen/Desktop/09_research/09_research_main/lab_03/data/NORMAL/G3_11_正常人_1234597A_M_55_20190122/G3_11_正常人_1234597A_M_55_20190122_jpg_CNN/../mask/jpgs/493_mask_frontglottis_1.jpg

In [ ]:
# frontglottis

def get_frontglottis(get_dir, get_file):
  file_name = get_file + '_mask_frontglottis_0.jpg'
  frontglottis_DIR = os.path.join(get_dir, 'frontglottis')
  print(os.path.join(frontglottis_DIR, file_name))
  img = cv2.imread(os.path.join(frontglottis_DIR, file_name))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # plt.imshow(img)
  # plt.show()
  return img

In [ ]:
# vfleft

def get_vfleft(get_dir, get_file):
  file_name = get_file + '_mask_vfleft_2.jpg'
  vfleft_DIR = os.path.join(get_dir, 'vfleft')
  # print(os.path.join(vfleft_DIR, file_name))
  img = cv2.imread(os.path.join(vfleft_DIR, file_name))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # plt.imshow(img)
  # plt.show()
  return img

In [ ]:
# vfright

def get_vfright(get_dir, get_file):
  file_name = get_file + '_mask_vfright_1.jpg'
  vfright_DIR = os.path.join(get_dir, 'vfright')
  # print(os.path.join(vfright_DIR, file_name))
  img = cv2.imread(os.path.join(vfright_DIR, file_name))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # plt.imshow(img)
  # plt.show()
  return img

In [ ]:
# 0 = ground

# 1 = frontglottis

# 2 = vfright_normal
# 3 = vfleft_normal

# 4 = vfright_rln_R
# 5 = vfleft_rln_R

# 6 = vfright_rln_L
# 7 = vfleft_rln_L

# 8 = vfright_rsln_R
# 9 = vfleft_rsln_R

# 10 = vfright_rsln_L
# 11 = vfleft_rsln_L

In [ ]:
# combine

def combine(mask_DIR, get_file, get_TYPE):
  mask_frontglottis = get_frontglottis(mask_DIR, get_file)
  mask_vfleft = get_vfleft(mask_DIR, get_file)
  mask_vfright = get_vfright(mask_DIR, get_file)
  # 0 = ground

  # 1 = frontglottis
  mask_frontglottis[mask_frontglottis > 0] = int(1)

  # 2 = vfright_normal
  # 3 = vfleft_normal
  if get_TYPE == 'NORMAL':
    mask_vfright[mask_vfright > 0] = int(2)
    mask_vfleft[mask_vfleft > 0] = int(3)
    print('now is ' + get_TYPE + ' type.')

  # 4 = vfright_rln_R
  # 5 = vfleft_rln_R
  if get_TYPE == 'RLN_R':
    mask_vfright[mask_vfright > 0] = int(4)
    mask_vfleft[mask_vfleft > 0] = int(5)
    print('now is ' + get_TYPE + ' type.')

  # 6 = vfright_rln_L
  # 7 = vfleft_rln_L
  if get_TYPE == 'RLN_L':
    mask_vfright[mask_vfright > 0] = int(6)
    mask_vfleft[mask_vfleft > 0] = int(7)
    print('now is ' + get_TYPE + ' type.')

  # 8 = vfright_rsln_R
  # 9 = vfleft_rsln_R
  if get_TYPE == 'RSLN_R':
    mask_vfright[mask_vfright > 0] = int(8)
    mask_vfleft[mask_vfleft > 0] = int(9)
    print('now is ' + get_TYPE + ' type.')

  # 10 = vfright_rsln_L
  # 11 = vfleft_rsln_L
  if get_TYPE == 'RSLN_L':
    mask_vfright[mask_vfright > 0] = int(10)
    mask_vfleft[mask_vfleft > 0] = int(11)
    print('now is ' + get_TYPE + ' type.')

  new_mask = np.concatenate([[mask_frontglottis],[mask_vfright],[mask_vfleft]]).max(axis=0)

  return new_mask

In [ ]:
# single_process

def single_process(get_dir, NEW_DATA_PATH, START_ID, TYPE):
  mask_DIR = os.path.join(get_dir, 'mask')
  jpgs_DIR = os.path.join(mask_DIR, 'jpgs')

  new_train_1_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_1_mask')
  checkpath(new_train_1_mask_DIR)

  new_train_1_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_1_jpgs')
  checkpath(new_train_1_jpgs_DIR)

  new_train_2_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_2_mask')
  checkpath(new_train_2_mask_DIR)

  new_train_2_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_2_jpgs')
  checkpath(new_train_2_jpgs_DIR)

  new_test_mask_DIR = os.path.join(NEW_DATA_PATH, 'test_mask')
  checkpath(new_test_mask_DIR)

  new_test_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'test_jpgs')
  checkpath(new_test_jpgs_DIR)

  counter = 0

  for get_file in os.listdir(jpgs_DIR):
    if get_file[-1] == 'g':

      if counter % 4 == 0:
        ima_mask_DIR = new_test_mask_DIR
        ima_jpgs_DIR = new_test_jpgs_DIR
      elif counter % 4 == 1:
        ima_mask_DIR = new_train_2_mask_DIR
        ima_jpgs_DIR = new_train_2_jpgs_DIR
      else:
        ima_mask_DIR = new_train_1_mask_DIR
        ima_jpgs_DIR = new_train_1_jpgs_DIR

      ima_ID = TYPE + str(START_ID) + '.png'

      combined_mask = combine(mask_DIR, get_file.split('.')[0], TYPE)

      mask_filename = os.path.join(ima_mask_DIR, ima_ID)
      print('Saving to ' + mask_filename + '\n')
      cv2.imwrite(mask_filename, combined_mask)

      pair_img = cv2.imread(os.path.join(jpgs_DIR, get_file))

      jpgs_filename = os.path.join(ima_jpgs_DIR, ima_ID)
      # print(jpgs_filename)
      cv2.imwrite(jpgs_filename, pair_img)

      START_ID += 1
      counter += 1

  return START_ID

In [ ]:
ROOT_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataP'

new_DATA_PATH =  os.path.join(ROOT_DIR, '../dataset_C_v_1.5.0')
checkpath(new_DATA_PATH)

In [ ]:
# # test
# NORMAL_DIR = os.path.join(ROOT_DIR, 'NORMAL')
# mask_DIR = os.path.join(NORMAL_DIR, 'mask')
# get_file = '10.jpg'
# TYPE = 'NORMAL'

# combined_mask = combine(mask_DIR, get_file.split('.')[0], TYPE)
# print(np.unique(combined_mask))

# mask_filename = os.path.join(ROOT_DIR, 'test.png')
# print(mask_filename)
# cv2.imwrite(mask_filename, combined_mask)

# img___ = cv2.imread(mask_filename)
# print(np.unique(img___))

In [ ]:
# all data checker

def data_checker(GET_ROOT_DIR):

  for get_folder_name in os.listdir(GET_ROOT_DIR):
    watching_this_dir = os.path.join(GET_ROOT_DIR, get_folder_name)
    print('Now checking in ' + watching_this_dir)

    jpgs_counter = 0
    mask_0_counter = 0
    mask_1_counter = 0
    mask_2_counter = 0

    mask_DIR = os.path.join(watching_this_dir, 'mask')
    jpgs_DIR = os.path.join(mask_DIR, 'jpgs')

    for get_jpgs_file in os.listdir(jpgs_DIR):
      if get_jpgs_file[-1] == 'g':
        jpgs_counter += 1

      for get_file in os.listdir(os.path.join(mask_DIR, 'frontglottis')):
        if get_file.replace('_mask_frontglottis_0.jpg', '') == get_jpgs_file.split('.')[0]:
          mask_0_counter += 1

      for get_file in os.listdir(os.path.join(mask_DIR, 'vfleft')):
        if get_file.replace('_mask_vfleft_2.jpg', '') == get_jpgs_file.split('.')[0]:
          mask_2_counter += 1

      for get_file in os.listdir(os.path.join(mask_DIR, 'vfright')):
        if get_file.replace('_mask_vfright_1.jpg', '') == get_jpgs_file.split('.')[0]:
          mask_1_counter += 1

    print(f'  {jpgs_counter} / {mask_0_counter} / {mask_1_counter} / {mask_2_counter}')
    if jpgs_counter == mask_0_counter and jpgs_counter == mask_1_counter and jpgs_counter == mask_2_counter:
      print('Check complete in ' + watching_this_dir + '\n')
    else:
      print('Having some problem in ' + watching_this_dir)

  print('\n\nCheck Completed Successfully!!!\n\n')


In [ ]:
data_checker(os.path.join(ROOT_DIR, 'test'))

In [ ]:
# main

data_checker(ROOT_DIR)

ima_ID = 0

for get_folder_name in os.listdir(ROOT_DIR):
  watching_this_dir = os.path.join(ROOT_DIR, get_folder_name)
  print('Now processing in ' + watching_this_dir)

  ima_ID = single_process(watching_this_dir, new_DATA_PATH, ima_ID, get_folder_name)

print('\n\nSuccessfully Completed!!!\n\n')


In [ ]:
# mask fit check

mask_DIR = os.path.join(new_DATA_PATH, 'train_1_mask')
jpgs_DIR = os.path.join(new_DATA_PATH, 'train_1_jpgs')


for get_file_name in os.listdir(jpgs_DIR):
  print('Now looking ' + get_file_name)

  mask = cv2.imread(os.path.join(mask_DIR, get_file_name))
  img = cv2.imread(os.path.join(jpgs_DIR, get_file_name))

  plt.imshow(img)
  plt.show()

  mask = mask*20
  plt.imshow(mask)
  plt.show()

  res = cv2.addWeighted(img, 0.5, mask, 0.5, 0)
  plt.imshow(res)
  plt.show()

In [ ]:
def prediction_map(get_list, pred_idx):
  imanopredict = []

  for i in range(0,12):
    imanopredict += [int(np.sum(np.asarray(pred_idx)==i))]

  sumR = 0
  sumL = 0

  for i in range(2, 12, 2):
    sumR += imanopredict[i]
    sumL += imanopredict[i+1]

  print('-'*25)
  for i in range(2, 12, 2):
    print(f'{imanopredict[i]/sumR*100:.2f}% = {get_list[i]}')

  print('-'*25)
  for i in range(3, 12, 2):
    print(f'{imanopredict[i]/sumL*100:.2f}% = {get_list[i]}')

  print('-'*25)

In [ ]:
import pandas as pd

In [ ]:
classescsv_path = os.path.join(new_DATA_PATH, "classes.csv")
classescsv = pd.read_csv(classescsv_path)
classes = list(classescsv['_class'])

In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask'

img_ = np.zeros(10)#622810

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  img_ = np.concatenate([img_, img])

prediction_map(classes, img_)

In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask'

img_ = np.zeros(10)#622810

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  img_ = np.concatenate([img_, img])

img_ = np.unique(img_)
print(img_)

In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask/0.png'
jpgs_DIR_ = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask/1039.png'


img = cv2.imread(os.path.join(jpgs_DIR, get_file))
img = np.unique(img)

img_ = cv2.imread(os.path.join(jpgs_DIR_, get_file))
img_ = np.unique(img_)

print(type(img[0]))
print(type(img_[0]))


In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask'

img_ = np.zeros(10)

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  print(get_file)
  print(img)